In [2]:
from tensorflow import keras
import tensorflow as tf

2023-01-14 12:39:37.383209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-14 12:39:37.780718: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-14 12:39:37.780780: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-14 12:39:40.756335: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
img_height,img_width = (96,96)
batch_size=32
data_dir = "./Bag Classes/Bag Classes/"
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 15000 files belonging to 3 classes.
Using 12000 files for training.


2023-01-14 12:39:46.997921: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-14 12:39:46.998020: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dimo-81ne): /proc/driver/nvidia/version does not exist
2023-01-14 12:39:47.002005: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 15000 files belonging to 3 classes.
Using 3000 files for validation.


In [4]:
class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

['Garbage Bag Images', 'Paper Bag Images', 'Plastic Bag Images']


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

In [6]:
train_ds = (
    train_ds
    .shuffle(1000)
    #.map(augment, num_parallel_calls=AUTOTUNE)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

val_ds = (
    val_ds
    # .map(rescale, num_parallel_calls=AUTOTUNE)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

base_model = tf.keras.applications.EfficientNetV2S(input_shape=(96,96,3), weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False

82420632/82420632 [==============================] - 15s 0us/step


In [7]:
#tf.keras.layers.GlobalMaxPooling2D
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(num_classes,activation = "softmax")
])

#experiments
# Flatten вместо GlobalAveragePooling - флатън дава по малко точност + добавя повече параметри
# По малкото лейъри носят повече скорост но по малка точност, а е важно тази нотификации да са точни
# 32,32 - доста ниска точност
# 64,64 - висока точност 0.87
# 96,96 - 

In [8]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [9]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)


Epoch 1/5


/home/dimoy/.local/lib/python3.10/site-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


375/375 [==============================] - 506s 1s/step - loss: 0.2459 - accuracy: 0.9050 - val_loss: 0.1810 - val_accuracy: 0.9317
Epoch 2/5
375/375 [==============================] - 585s 2s/step - loss: 0.1446 - accuracy: 0.9464 - val_loss: 0.1562 - val_accuracy: 0.9410
Epoch 3/5
375/375 [==============================] - 288s 769ms/step - loss: 0.1048 - accuracy: 0.9615 - val_loss: 0.1586 - val_accuracy: 0.9420
Epoch 4/5
375/375 [==============================] - 216s 577ms/step - loss: 0.0767 - accuracy: 0.9726 - val_loss: 0.1641 - val_accuracy: 0.9420
Epoch 5/5
375/375 [==============================] - 21402s 57s/step - loss: 0.0605 - accuracy: 0.9797 - val_loss: 0.1690 - val_accuracy: 0.9427


In [10]:
model.save("EfficientNetModelBagRecognition")

INFO:tensorflow:Assets written to: EfficientNetModelBagRecognition/assets


INFO:tensorflow:Assets written to: EfficientNetModelBagRecognition/assets


In [ ]:
model.fit(data)

160/469 [=========>....................] - ETA: 10s - loss: -4558.2930 - binary_accuracy: 0.3373 - false_negatives_1: 0.0000e+00

KeyboardInterrupt: 